<a href="https://colab.research.google.com/github/farhadkamangar/CSE5368/blob/master/tensorflow_multi_layer_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
class DenseLayer(tf.Module):
  def __init__(self, input_dimension, number_of_nodes, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([input_dimension, number_of_nodes]), name='weights')
    self.b = tf.Variable(tf.zeros([number_of_nodes]), name='biases')
  def calculate_output(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [ ]:
class MultiLayer_NN(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.layer_1 = DenseLayer(input_dimension=3, number_of_nodes=3)
    self.layer_2 = DenseLayer(input_dimension=3, number_of_nodes=6)

  def predict(self, x):
    x = self.layer_1.calculate_output(x)
    return self.layer_2.calculate_output(x)

my_model = MultiLayer_NN(name="the_model")

output_of_model=my_model.predict(tf.constant([[1.0, 2.0, 3.0]]))
print("Model results:", output_of_model)
print("Weights ", my_model.layer_2.w)
print("Submodules:", my_model.submodules)
for var in my_model.variables:
  print(var, "\n")

Model results: tf.Tensor([[0.        0.        2.0554593 0.        1.7507195 3.9516466]], shape=(1, 6), dtype=float32)
Weights  <tf.Variable 'weights:0' shape=(3, 6) dtype=float32, numpy=
array([[-1.2040013 , -0.41027942,  0.4943638 , -1.9943864 ,  0.7568495 ,
         0.47564486],
       [ 1.1751459 , -0.06662586,  1.5454153 , -0.705252  , -0.84897923,
         0.07810022],
       [-0.9712359 , -1.5651697 ,  0.5366369 , -0.0289091 , -0.21630858,
         1.9838241 ]], dtype=float32)>
Submodules: (<__main__.DenseLayer object at 0x7f81d08c7110>, <__main__.DenseLayer object at 0x7f81d08c7f10>)
<tf.Variable 'biases:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'weights:0' shape=(3, 3) dtype=float32, numpy=
array([[ 1.1812481 , -0.41367558, -1.0155593 ],
       [ 0.8020408 , -0.83198446,  0.15475112],
       [-0.02923901,  0.2549367 ,  0.6837357 ]], dtype=float32)> 

<tf.Variable 'biases:0' shape=(6,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0

In [ ]:
class FlexibleDenseLayer(tf.Module):
  def __init__(self, number_of_nodes, name=None):
    super().__init__(name=name)
    self.first_time=True
    self.number_of_nodes=number_of_nodes
    
  def calculate_output(self, x):
    if self.first_time:
      self.w = tf.Variable(tf.random.normal([x.shape[-1], self.number_of_nodes]), name='weights')
      self.b = tf.Variable(tf.zeros([self.number_of_nodes]), name='biases')
      self.first_time=False
    y = tf.matmul(x, self.w) + self.b
    # return tf.nn.hard_tanh(y)
    # return tf.nn.relu(y)
    return y

In [ ]:
class MultiLayer_NN(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.layer_1 = FlexibleDenseLayer(number_of_nodes=3)
    self.layer_2 = FlexibleDenseLayer(number_of_nodes=5)

  def predict(self, x):
    x = self.layer_1.calculate_output(x)
    return self.layer_2.calculate_output(x)

my_model = MultiLayer_NN(name="the_model")

output_of_model=my_model.predict(tf.constant([[1.0, 2.0, 3.0]]))
print("Model results:", output_of_model)
print("Weights ", my_model.layer_2.w)
print("Submodules:", my_model.submodules)
for var in my_model.variables:
  print(var, "\n")

Model results: tf.Tensor([[-1.8583658  4.2238026  1.9643538 -0.4884634  4.536718 ]], shape=(1, 5), dtype=float32)
Weights  <tf.Variable 'weights:0' shape=(3, 5) dtype=float32, numpy=
array([[-0.1492263 ,  0.9969619 , -0.46697536, -0.05895721, -0.4458336 ],
       [-0.48961434, -0.8972078 , -1.9212656 , -0.6093035 , -1.3387625 ],
       [ 1.8825271 , -2.0720181 ,  1.570494  ,  1.1322998 , -0.79636824]],
      dtype=float32)>
Submodules: (<__main__.FlexibleDenseLayer object at 0x7f1eecfd2750>, <__main__.FlexibleDenseLayer object at 0x7f1eecfd2250>)
<tf.Variable 'biases:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'weights:0' shape=(3, 3) dtype=float32, numpy=
array([[-0.12552713, -0.13028823, -1.7574427 ],
       [-0.52977663,  0.8169243 ,  0.9981878 ],
       [ 0.15518194, -1.7350601 , -0.04664988]], dtype=float32)> 

<tf.Variable 'biases:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)> 

<tf.Variable 'weights:0' sha

In [ ]:
chkp_path = "my_checkpoint_01"
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)

'my_checkpoint_01'

In [ ]:
ls my_checkpoint*

my_checkpoint_01.data-00000-of-00001  my_checkpoint.data-00000-of-00001
my_checkpoint_01.index                my_checkpoint.index


In [ ]:
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/layer_1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/layer_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/layer_2/b/.ATTRIBUTES/VARIABLE_VALUE', [6]),
 ('model/layer_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 6])]

In [ ]:
new_model = MultiLayer_NN()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("my_checkpoint")

for var in my_model.variables:
  print(var, "\n")

<tf.Variable 'biases:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'weights:0' shape=(3, 3) dtype=float32, numpy=
array([[ 1.1812481 , -0.41367558, -1.0155593 ],
       [ 0.8020408 , -0.83198446,  0.15475112],
       [-0.02923901,  0.2549367 ,  0.6837357 ]], dtype=float32)> 

<tf.Variable 'biases:0' shape=(6,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0.], dtype=float32)> 

<tf.Variable 'weights:0' shape=(3, 6) dtype=float32, numpy=
array([[-1.2040013 , -0.41027942,  0.4943638 , -1.9943864 ,  0.7568495 ,
         0.47564486],
       [ 1.1751459 , -0.06662586,  1.5454153 , -0.705252  , -0.84897923,
         0.07810022],
       [-0.9712359 , -1.5651697 ,  0.5366369 , -0.0289091 , -0.21630858,
         1.9838241 ]], dtype=float32)> 

